In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
%load_ext nb_mypy

In [ ]:
import nbimporter
from AIBaseClass import ChessAI
from Exercise05AI import Exercise05AI
from Exercise04AI import Exercise04AI

# Aufgabe 06: Minimax mit Alpha-Beta-Pruning und Memoisierung

Dieses Notebook implementiert den Minimax-Algorithmus mit Alpha-Beta-Pruning und Memoisierung.

In [ ]:
import chess_custom as chess
from typing import Any

class Exercise06AI(Exercise05AI):
    """Chooses middle game moves using minimax algorithm, alpha-beta-pruning and memoization"""
    def __init__(self, **kwargs) -> None:
        super().__init__(**kwargs)
        self.cache: dict[tuple, Any] = {}
        self.cache_hits = 0
    
    def reset(self) -> None:
        """Resets all internal variables"""
        super().reset()
        self.cache = {}
        self.cache_hits = 0

## Memoisierungsfunktion

Die Klasse wird um eine Variable `cache` erweitert. Initial ist `cache` ein leeres Dictionary, welches als Schlüssel ein Tupel annimmt und als Wert ein beliebiges Objekt zurückgibt.

Die Memoisierungsfunktion `memoize_minimax` nimmt die Minimax-Funktion `minimax` als Argument und gibt eine memoisierte Version `minimax_memoized` dieser Funktion zurück.

Die memoisierte Funktion `minimax_memoized` versucht zunächst, den Rückgabewert der Ursprungsfunktion aus dem `cache` auszulesen. Dazu wird die Hilsfunktion `get_from_cache` verwendet. Beinhaltet der `cache` ein Ergebnis für die gegebenen Argumente `args`, so wird dieses zurückgegeben. Andernfalls wird die Ursprungsfunktion `minimax` aufgerufen, der Rückgabewert wird für die verwendeten Argumente im `cache` hinterlegt und schlussendlich wird der berechnete Wert zurückgegeben. Zum Speichern des neuen Rückgabewertes wird die Hilfsfunktion `store_in_cache` verwendet.

### Cache-Verwaltung

Im Vergleich zu `Exercise04AI` sind die Argumente $alpha$ und $beta$, welche für das Alpha-Beta-Pruning benötigt werden, zum Funktionsaufruf der Methode `minimax` hinzugekommen. Demzufolge wird die Interaktion mit dem Cache überarbeitet.

#### Speicherung im Cache

Bisher war `result` ein Tupel der Form $$result = (evaluation, move)$$
Um Alpha-Beta-Pruning memoisieren zu können, wird nun nicht mehr `result` im Cache gespeichert, sondern ein Tripel der Form $$ value = (flag, evaluation, move) $$ mit $$ flag \in \{ \; \text{'≤'}, \text{'='}, \text{'≥'} \; \} $$

wobei gilt $$flag = \begin{cases}
\text{'≤'} \; \text{wenn} \; evaluation \le alpha\\
\text{'='} \; \text{wenn} \; evaluation < beta\\
\text{'≥'} \; \text{sonst}
\end{cases}$$

Das Speichern dieses Tupels wird von der Funktion `store_in_cache` übernommen.

#### Abrufen aus dem Cache

Die Funktion `get_from_cache` nimmt als Parameter die Funktion `minimax`, das Tupel `key`, sowie alle Parameter der Funktion `minimax` entgegen und gibt ein Tupel der Form `result` zurück. Zunächst wird das Tripel der Form $$ value = (flag, evaluation, move) $$ mit dem gegebenen `key` aus dem Cache extrahiert. Es folgt eine Fallunterscheidung nach dem Wert von $flag$:

1. Fall: $flag = \text{'='}$

    In diesem Fall wird das Tupel $(evaluation, move)$ zurückgegeben.

2. Fall: $flag = \text{'≤'}$

    Ist $evaluation \le alpha$, so kann das Tupel $(evaluation, move)$ direkt zurückgegeben werden. Ist $evaluation < beta$, so wird die Funktion `minimax` mit $alpha$ und $evaluation$ aufgerufen, andernfalls gilt $evaluation \ge beta$ und die Funktion `minimax` wird mit den ursprünglichen Grenzen $alpha$ und $beta$ aufgerufen. Das Ergebnis wird im Cache gespeichert und zurückgegeben.

3. Fall: Trifft keiner der vorherigen Fälle zu, muss $flag = \text{'≥'}$ gelten und es folgt eine erneute Fallunterscheidung:

    Ist $evaluation \le alpha$, so wird die Funktion `minimax` mit $alpha$ und $beta$ aufgerufen. Ist $evaluation < beta$, so wird `minimax` stattdessen mit $evaluation$ und $beta$ aufgerufen. Das Ergebnis wird jeweils im Cache gespeichert und zurückgegeben.

    Trifft keiner der beiden Fälle zu, muss $evaluation > beta$ gelten und das Tupel $(evaluation, move)$ kann direkt zurückgegeben werden.


In [ ]:
from typing import Any, Callable

class Exercise06AI(Exercise06AI): # type: ignore
    
    @staticmethod
    def memoize_minimax(minimax: Callable):
        def minimax_memoized(self, board: chess.Board, depth: int, current_eval: int,
                             alpha: int = -Exercise06AI.LIMIT, beta: int = Exercise06AI.LIMIT):
            key = Exercise04AI.get_key(board, depth)
            if key in self.cache:
                self.cache_hits += 1
                return self.get_from_cache(minimax, key, board, depth, current_eval, alpha, beta)
            result = minimax(self, board, depth, current_eval, alpha, beta)
            self.store_in_cache(key, result, alpha, beta)
            return result

        return minimax_memoized
    
    @memoize_minimax
    def minimax(self, *args) -> int:
        """Memoized version of the Exercise05AI minimax implementation"""
        return super().minimax(*args)
    
    def store_in_cache(self, key: tuple, result: tuple, alpha: int, beta: int) -> None:
        """Stores the result of a minimax computation in the cache."""
        evaluation, move = result
        if evaluation <= alpha:
            self.cache[key] = ("≤", evaluation, move)
        elif evaluation < beta:
            self.cache[key] = ("=", evaluation, move)
        else:
            self.cache[key] = ("≥", evaluation, move)
            
    def get_from_cache(self, minimax: Callable, key: tuple, board: chess.Board,
                       depth: int, current_eval: int, alpha: int, beta: int) -> tuple:
        """Gets a result from the cache if possible."""
        flag, evaluation, move = self.cache[key]
        if flag == "=":
            return evaluation, move
        elif flag == "≤":
            if evaluation <= alpha:
                return evaluation, move
            elif evaluation < beta:
                result = minimax(self, board, depth, current_eval, alpha, evaluation)
                self.store_in_cache(key, result, alpha, evaluation)
                return result
            else:
                result = minimax(self, board, depth, current_eval, alpha, beta)
                self.store_in_cache(key, result, alpha, beta)
                return result
        else:
            if evaluation <= alpha:
                result = minimax(self, board, depth, current_eval, alpha, beta)
                self.store_in_cache(key, result, alpha, beta)
                return result
            elif evaluation < beta:
                result = minimax(self, board, depth, current_eval, evaluation, beta)
                self.store_in_cache(key, result, evaluation, beta)
                return result
            else:
                return evaluation, move

## Debugging Bereich

Die folgenden Zellen enthalten Unit-Tests der oben implementierten Funktionen.

In [ ]:
# Create player and board
unit_test_player = Exercise06AI(player_name="Ex06AI", search_depth=2)
board = chess.Board("5rk1/1b3p2/8/3p4/3p2P1/2Q4B/5P1K/R3R3 b - - 0 36")
next_move = chess.Move.from_uci('d4c3') # white queen capture
board

In [ ]:
# Test cache store function
unit_test_player.cache = {}  # Clear cache
test_result = (10, next_move)

test_key_1 = ("test", 1)
unit_test_player.store_in_cache(test_key_1, test_result, alpha=5, beta=15)
assert unit_test_player.cache[test_key_1] == ('=', *test_result), "Cache result does not match expected value!"

test_key_2 = ("test", 2)
unit_test_player.store_in_cache(test_key_2, test_result, alpha=12, beta=15)
assert unit_test_player.cache[test_key_2] == ('≤', *test_result), "Cache result does not match expected value!"

test_key_3 = ("test", 3)
unit_test_player.store_in_cache(test_key_3, test_result, alpha=5, beta=8)
assert unit_test_player.cache[test_key_3] == ('≥', *test_result), "Cache result does not match expected value!"

unit_test_player.cache

In [ ]:
# Test minimax
unit_test_player.cache = {}  # Clear cache
evaluation = unit_test_player.full_evaluate(board)  # Get current evaluation
mm_evaluation, mm_move = unit_test_player.minimax(board, 2, evaluation)
assert mm_evaluation == 355, "Minimax evaluation does not match expected value!"
assert mm_move.uci() == 'd4c3', "Minimax move does not match expected value!"
assert unit_test_player.cache != {}, "Cache is empty!"
mm_evaluation, mm_move, len(unit_test_player.cache)

In [ ]:
# Test minimax again (now memoized)
unit_test_player.cache_hits = 0  # Reset cache hits counter
mm_evaluation, mm_move = unit_test_player.minimax(board, 2, evaluation)
assert mm_evaluation == 355, "Minimax evaluation does not match expected value!"
assert mm_move.uci() == 'd4c3', "Minimax move does not match expected value!"
assert unit_test_player.cache_hits == 1, "Cache was not used correctly!"
mm_evaluation, mm_move, unit_test_player.cache_hits

In [ ]:
# Test next move function (with memoized minimax result)
move = unit_test_player.get_next_middle_game_move(board)
assert move.uci() == 'd4c3', "Next move does not match expected value!"
move

In [ ]:
# Test reset function
unit_test_player.reset()
assert unit_test_player.last_evaluation is None, "Reset was not successful! (last_evaluation)"
assert unit_test_player.is_king_endgame is False, "Reset was not successful! (is_king_endgame)"
assert unit_test_player.tables == unit_test_player.PIECE_TO_PST, "Reset was not successful! (tables)"
assert unit_test_player.cache == {}, "Reset was not successful! (cache)"
assert unit_test_player.cache_hits == 0, "Reset was not successful! (cache_hits)"

## Temporärer Bereich

Der folgende Bereich dient zum temporären Debuggen und kann nicht-funktionierenden Code enthalten. Dieser Bereich wird vor der Abgabe entfernt.

In [ ]:
#debug
from Exercise03AI import Exercise03AI

In [ ]:
#debug
board = chess.Board("5rk1/1b3p2/8/3p4/3p2P1/2Q4B/5P1K/R3R3 w - - 0 36")
board.push(chess.Move.from_uci("h2h1"))
board

In [ ]:
%%time
DEPTH = 4
player3 = Exercise03AI("Testplayer", DEPTH)
#print(player.minimax(board, DEPTH))
move = player3.get_next_middle_game_move(board)
print(move)

In [ ]:
%%time
DEPTH = 4
player5 = Exercise05AI("Testplayer", DEPTH)
#print(player.minimax(board, DEPTH))
move = player5.get_next_middle_game_move(board)
print(move)

In [ ]:
%%time
DEPTH = 4
player6 = Exercise06AI("Testplayer", DEPTH)
#print(player.minimax(board, DEPTH))
move = player6.get_next_middle_game_move(board)
print(move)
print(player6.cache_hits)

In [ ]:
#debug
from IPython.display import clear_output, display
player5 = Exercise05AI("Testplayer", 3)
player3 = Exercise03AI("Testplayer", 3)
move = player3.random.choice(list(board.legal_moves))
board.push(move)
for _ in range(10):
    move_1, eval_1 = player3.get_next_middle_game_move(board)
    player3.last_evaluation = None
    print(move_1, eval_1, player3.last_evaluation)
    move_2, eval_2 = player5.get_next_middle_game_move(board)
    player5.last_evaluation = None
    print(move_2, eval_2, player5.last_evaluation)
    move = player3.random.choice(list(board.legal_moves))
    board.push(move)

In [ ]:
%%time
DEPTH = 3
board = chess.Board()
#board.set_fen("4k3/8/2n5/7K/5q2/2N5/8/2B5 b - - 0 1")
player = Exercise04AI("Testplayer", DEPTH)
for _ in range(10):
    board.push(sorted(board.legal_moves, key=lambda move: move.uci())[0])
    move = player.get_next_middle_game_move(board)
    print(move)

In [ ]:
#debug
from os.path import join

with open(join("..", "games", "2022-01-15_20-18-57-907777.pgn")) as pgn:
    first_game = chess.pgn.read_game(pgn)

# Iterate through all moves and play them on a board.
board = first_game.board()
for move in first_game.mainline_moves():
    board.push(move)
for i in range(220):
    board.pop()

In [ ]:
%%time
DEPTH = 2
#board = chess.Board()
#board.set_fen("4k3/8/2n5/7K/5q2/2N5/8/2B5 b - - 0 1")
player = Exercise04AI("Testplayer", DEPTH)
#print(player.minimax(board, DEPTH))
move = player.get_next_middle_game_move(board)
print(move)
player.cache_hits

In [ ]:
#debug
board = chess.Board("7B/pbpk4/1p6/2n1pp2/6p1/4P3/P1P2PNP/RNq1K1R1 w - - 0 1")
player = Exercise04AI("Testplayer", 3)
print(player.evaluate(board))
board